In [40]:
!python -V

Python 3.10.12


In [41]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer as dv
from sklearn.linear_model import LinearRegression as lr
from sklearn.metrics import mean_squared_error

In [42]:
ny_jan_df = pd.read_parquet("./yellow_tripdata_2023-01.parquet")
ny_feb_df = pd.read_parquet("./yellow_tripdata_2023-02.parquet")


# **Q1. Downloading the data**

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [43]:
len(ny_jan_df.columns)

19


# **Q2. Computing duration**

Now let's compute the duration variable. It should contain the duration of a ride in minutes.
What's the standard deviation of the trips duration in January?


In [44]:
ny_jan_df["duration"] = ny_jan_df.tpep_dropoff_datetime - ny_jan_df.tpep_pickup_datetime

ny_jan_df.duration = ny_jan_df.duration.apply(lambda x:x.total_seconds()/60)

ny_jan_df.duration.std(skipna=True)

42.59435124195458

# **Q3. Dropping outliers**

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [45]:
rec_count_before = len(ny_jan_df)
ny_jan_df = ny_jan_df[(ny_jan_df.duration >= 1) & (ny_jan_df.duration <= 60)]
rec_count_after = len(ny_jan_df)
print("Fraction of record count left after dropping outlier: %0.2f" % (rec_count_after/rec_count_before))

Fraction of record count left after dropping outlier: 0.98


# **Q4. One-hot encoding**

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [46]:
categorical = ['PULocationID','DOLocationID']
ny_jan_df[categorical] = ny_jan_df[categorical].astype(str)

dict_train = ny_jan_df[['PULocationID','DOLocationID', 'trip_distance']].to_dict(orient='records')

dv1 = dv()  #create an instance of DictVectorizer()
xtrain = dv1.fit_transform(dict_train)



In [47]:
len(dv1.feature_names_)

516

# **Q5. Training a model**

Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train?

In [48]:
ytrain = ny_jan_df['duration'].values
ytrain

array([ 8.43333333,  6.31666667, 12.75      , ..., 24.51666667,
       13.        , 14.4       ])

In [49]:
lr1 = lr()
lr1.fit(xtrain, ytrain)


LinearRegression()

In [50]:
ypredict = lr1.predict(xtrain)

In [51]:
rmse_train = np.sqrt(mean_squared_error(ytrain, ypredict))
print("RMSE on train: ", rmse_train)

RMSE on train:  7.656752063242953


# **Q6. Evaluating the model**

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [52]:
ny_feb_df["duration"] = ny_feb_df.tpep_dropoff_datetime - ny_feb_df.tpep_pickup_datetime

ny_feb_df.duration = ny_feb_df.duration.apply(lambda x:x.total_seconds()/60)
ny_feb_df = ny_feb_df[(ny_feb_df.duration >= 1) & (ny_feb_df.duration <= 60)]

yval = ny_feb_df['duration'].values

ny_feb_df[categorical] = ny_feb_df[categorical].astype(str)

dict_val = ny_feb_df[['PULocationID','DOLocationID', 'trip_distance']].to_dict(orient='records')
#dv2 = dv()  #create an instance of DictVectorizer()
xval = dv1.transform(dict_val)

ypredict = lr1.predict(xval)

rmse_val = np.sqrt(mean_squared_error(yval, ypredict))
print("RMSE on validation data: ", rmse_val)


RMSE on validation data:  7.818628965425388


In [53]:
ny_jan_df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                     object
DOLocationID                     object
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
duration                        float64
dtype: object